# Checkpoint 1 Code

By: Gloria Kao, Shentong Li

Outputs (tables, aggregated data, graphs, etc.) are commented out and not shown because of NDA.

## 1. EDA

In [ ]:
import numpy as np 
import pandas as pd
# pacakges for geospatial analysis and plotting
import geopandas as gpd
import folium
import seaborn as sns
from shapely.geometry import Point
import matplotlib.pyplot as plt
from folium.plugins import HeatMap

We have 5 datasets in total. We focus on 3 of them first: 

1. `gis_weatherstation_shape_2024_10_04.csv`: Information of weather stations such as names, location, structure details, etc.
2. `src_wings_meteorology_station_summary_snapshot_2023_08_02.csv`: Meteorology data for each weather stations such as max gust and alert windspeed. 
3. `src_wings_meteorology_windspeed_snapshot_2023_08_02.csv`: Windspeed snapshots collected from weather stations, ranging from years 2012 to 2022. 

In [2]:
gis_2024_1004 = pd.read_csv('data/gis_weatherstation_shape_2024_10_04.csv')
station_summary_2023_08_02 = pd.read_csv('data/src_wings_meteorology_station_summary_snapshot_2023_08_02.csv')
windspeed_2023_08_02 = pd.read_csv('data/src_wings_meteorology_windspeed_snapshot_2023_08_02.csv')

### 1.1 Table 1 - GIS 2024_10_04
#### 1.1.1 Basic Summary Stats

In [ ]:
gis_2024_1004

In [ ]:
gis_2024_1004.columns

In [ ]:
station_location = gis_2024_1004[['weatherstationcode', 'latitude', 'longitude']]
station_location

In [ ]:
gis_2024_1004.describe()

In [ ]:
# check null
gis_2024_1004.isnull().sum()

In [ ]:
# num of stations contained
gis_2024_1004['weatherstationname'].nunique()

In [ ]:
duplicate_stations = gis_2024_1004[gis_2024_1004.duplicated(subset=['weatherstationname'], keep=False)]
duplicate_stations

In [ ]:
# count of each values in 'nwszone'
gis_2024_1004['nwszone'].value_counts()

#### 1.1.2 Geospatial Analysis
Show the details of each station by clicking on the icon in the map.

In [ ]:
map_center = [gis_2024_1004['latitude'].mean(), gis_2024_1004['longitude'].mean()]
m1 = folium.Map(location=map_center, zoom_start=10)

# Add weather station points to the map
for _, row in gis_2024_1004.iterrows():
    # Create a popup with relevant information
    popup_text = f"""
    Weather Station: {row['weatherstationname']}<br>
    Elevation: {row['elevation']} m<br>
    NWS Zone: {row['nwszone']}<br>
    Structure ID: {row['structureid']}<br>
    """
    
    # Add a marker for each weather station
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m1)


boundary_coords = [
    (gis_2024_1004['latitude'].min(), gis_2024_1004['longitude'].min()),
    (gis_2024_1004['latitude'].min(), gis_2024_1004['longitude'].max()),
    (gis_2024_1004['latitude'].max(), gis_2024_1004['longitude'].max()),
    (gis_2024_1004['latitude'].max(), gis_2024_1004['longitude'].min())
]

# boundary box
# folium.Polygon(locations=boundary_coords, color='green', fill=True, fill_opacity=0.2).add_to(m1)

m1.save('weather_stations_with_area_map.html')
m1

### 1.2 Table 2 - Station Summary 2023_08_02
#### 1.2.1 Basic Summary Statistics

In [ ]:
station_summary_2023_08_02

In [ ]:
station_summary_2023_08_02.describe()

In [ ]:
# distribution graphs 
sns.histplot(station_summary_2023_08_02['max_gust'], bins=10, kde=True)
plt.title('Distribution of Maximum Gusts')
plt.xlabel('Max Gust (mph)')
plt.ylabel('Frequency')
plt.show()
# plt.close()


sns.histplot(station_summary_2023_08_02['99th'], bins=10, kde=True)
plt.title('Distribution of 99th Percentile Gusts')
plt.xlabel('99th Percentile Gust (mph)')
plt.ylabel('Frequency')
plt.show()
# plt.close()


sns.histplot(station_summary_2023_08_02['95th'], bins=10, kde=True)
plt.title('Distribution of 95th Percentile Gusts')
plt.xlabel('99th Percentile Gust (mph)')
plt.ylabel('Frequency')
plt.show()
# plt.close()

In [ ]:
sns.countplot(x='vri', data=station_summary_2023_08_02)
plt.title('VRI (Risk Classification) Distribution')
plt.xlabel('VRI (H = High, M = Medium, L = Low)')
plt.ylabel('Count of Stations')
plt.show()
# plt.close()

# Bar plot for Alert Levels
sns.countplot(x='alert', data=station_summary_2023_08_02)
plt.title('Alert Level Distribution')
plt.xlabel('Alert Level')
plt.ylabel('Count of Stations')
plt.show()
# plt.close()

### 1.2.2 Merging datasets

In [ ]:
# merging the two datasets about weather stations together
merged_df = pd.merge(station_summary_2023_08_02, gis_2024_1004, right_on= 'weatherstationcode', left_on='station', how='left')
merged_df

In [ ]:
bins = range(0, 5800, 400)  
labels = [f'Group{i+1}: {bins[i]}-{bins[i+1]}' for i in range(len(bins)-1)]  # Create group labels

# Assign the binned elevation groups
merged_df['elevation_group'] = pd.cut(merged_df['elevation'], bins=bins, labels=labels)


elevation_vri_grouped = merged_df.groupby('elevation_group')['vri'].value_counts().unstack().fillna(0)
elevation_vri_grouped.plot(kind='bar', stacked=True, cmap='viridis')
plt.title('VRI (Risk Classification) Across Elevation Groups', fontsize=14)
plt.xlabel('Elevation Groups', fontsize=12)
plt.ylabel('Number of Stations', fontsize=12)
plt.legend(title='VRI Levels', loc='upper right')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
# plt.close()

C:\Users\glori\AppData\Local\Temp\ipykernel_15384\1844995259.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  elevation_vri_grouped = merged_df.groupby('elevation_group')['vri'].value_counts().unstack().fillna(0)


In [ ]:
vri_weights = {'H': 3, 'M': 2, 'L': 1}
merged_df['vri_weight'] = merged_df['vri'].map(vri_weights)

# Check for missing values and remove rows with NaN in latitude, longitude, or vri_weight
cleaned_df = merged_df.dropna(subset=['latitude', 'longitude', 'vri_weight'])

# Create a list of [latitude, longitude, weight] for the heatmap
heat_data = [[row['latitude'], row['longitude'], row['vri_weight']] for index, row in cleaned_df.iterrows()]

# Create a folium map centered around the average coordinates of the data
m = folium.Map(location=[cleaned_df['latitude'].mean(), cleaned_df['longitude'].mean()], zoom_start=10)

# Add the heatmap layer
HeatMap(heat_data, min_opacity=0.2, radius=20, blur=15, max_zoom=1).add_to(m)

# Save the map to an HTML file and display it
m.save('geospatial_risk_heatmap.html')

# If running in Jupyter or similar environments, you can display the map directly
m

### 1.3 Table 3 - Windspeed 2023_08_02
#### 1.3.1 Basic Summary Stat

In [ ]:
windspeed_2023_08_02_edit = windspeed_2023_08_02.reset_index().drop(columns=['index'])
windspeed_2023_08_02_edit['date'] = pd.to_datetime(windspeed_2023_08_02_edit['date'], format='%m/%d/%Y')
windspeed_2023_08_02_edit

In [ ]:
station_summary = windspeed_2023_08_02_edit.groupby('station')['wind_speed'].describe()
station_summary_edit = station_summary.reset_index()
station_summary_edit

In [ ]:
location_wind_speed_merge = pd.merge(station_location, station_summary, left_on='weatherstationcode', right_on='station', how='right')
location_wind_speed_merge_edit = location_wind_speed_merge.drop(columns=['weatherstationcode'])
location_wind_speed_merge_edit

In [ ]:
matrix = location_wind_speed_merge_edit.corr()
sns.heatmap(matrix, cmap="Greens", annot=True)

> Seems that there is a correlation between the wind speed and the longitude. 

In [ ]:
windspeed_2023_08_02_group = windspeed_2023_08_02.groupby('station')['wind_speed'].mean()
windspeed_2023_08_02_group

In [ ]:
# Histogram for wind speed distribution
plt.figure(figsize=(10, 6))
sns.histplot(windspeed_2023_08_02_edit['wind_speed'], bins=20, kde=True)
plt.title('Distribution of Wind Speeds')
plt.xlabel('Wind Speed (mph)')
plt.ylabel('Frequency')
plt.show()
# plt.close()

> We have an outlier of windspeed over 600mph.

In [ ]:
windspeed_2023_08_02[windspeed_2023_08_02['wind_speed'] > 600]

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='station', y='wind_speed', data=windspeed_2023_08_02_edit)
plt.title('Wind Speed Distribution by Station')
plt.xticks(rotation=90)
plt.xlabel('Station')
plt.ylabel('Wind Speed (mph)')
plt.show()
# plt.close()

#### 1.3.2 Analysis of windspeed over time 

In [ ]:
windspeed_2023_08_02_edit['month'] = windspeed_2023_08_02_edit['date'].dt.month

month_summary = windspeed_2023_08_02_edit.groupby('month')['wind_speed'].describe()
month_summary

In [ ]:
plt.figure(figsize=(14, 6))
sns.lineplot(x='date', y='wind_speed', data=windspeed_2023_08_02_edit)
plt.title('Wind Speed Over Time (All Stations)')
plt.xlabel('Date')
plt.ylabel('Wind Speed (mph)')
plt.show()
# plt.close()

In [ ]:
windspeed_2023_08_02_edit['date'] = pd.to_datetime(windspeed_2023_08_02_edit['date'])

# Extract month and year from the date
windspeed_2023_08_02_edit['month'] = windspeed_2023_08_02_edit['date'].dt.month
windspeed_2023_08_02_edit['year'] = windspeed_2023_08_02_edit['date'].dt.year

# Boxplot to show wind speed by month
plt.figure(figsize=(12, 6))
sns.boxplot(x='month', y='wind_speed', data=windspeed_2023_08_02_edit)
plt.title('Wind Speed by Month')
plt.xlabel('Month')
plt.ylabel('Wind Speed (mph)')
plt.show()
# plt.close()

In [ ]:
seasonal_corr = windspeed_2023_08_02_edit.groupby('month')['wind_speed'].mean()

# Plot the average wind speed for each month
plt.figure(figsize=(12, 6))
seasonal_corr.plot(kind='bar')
plt.title('Average Wind Speed by Month')
plt.xlabel('Month')
plt.ylabel('Average Wind Speed (mph)')
plt.show()
# plt.close()

## 2. Probability

Calculating PSPS Probability for each Weather Station 

In [ ]:
# not all stations have the same number of windspeed records
windspeed_grouped_count = windspeed_2023_08_02.groupby(by='station').count()
windspeed_grouped_count

In [32]:
station_codes = np.array(gis_2024_1004['weatherstationcode'])
merged_station_df = gis_2024_1004.merge(station_summary_2023_08_02, left_on='weatherstationcode', right_on='station', how='left')

In [ ]:
# example: showing the windspeed alert threshold for the station "AMO"
merged_df[merged_df['weatherstationcode']=='AMO']['alert'].iloc[0]

35

In [ ]:
# getting the PSPS probabilities of all weather stations
prob_lst = []

for station in station_codes:
    station_windspeeds = np.array(windspeed_2023_08_02[windspeed_2023_08_02['station'] == station]['wind_speed'])
    # "alert" might be nan because of less entries in station_ss_df 
    has_threshold = True
    try: 
        threshold = merged_df[merged_df['weatherstationcode'] == station]['alert'].iloc[0]
    except:
        has_threshold = False
        prob = np.nan
    mean = np.nanmean(station_windspeeds)
    if has_threshold:
        prob = np.mean([1 if x >= threshold else 0 for x in station_windspeeds]) * 100
    count = np.count_nonzero(~np.isnan(station_windspeeds))
    prob_lst.append([station, station_windspeeds, threshold, count, mean, prob])

C:\Users\glori\AppData\Local\Temp\ipykernel_15384\2284884836.py:12: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(station_windspeeds)


In [ ]:
# viewing the probabilities as a dataframe
prob_df = pd.DataFrame(prob_lst)
prob_df.columns = ['station', 'windspeeds', 'threshold', 'count', 'mean', 'probability (%)']
prob_df

In [36]:
print('max prob: ' + str(prob_df['probability (%)'].max()))
print('min prob: ' + str(prob_df['probability (%)'].min()))

max prob: 85.47486033519553
min prob: 0.0


In [37]:
# station mismatches between table 1 and table 2
prob_mismatch = prob_df[prob_df['count'] == 0]

In [38]:
# sort probability high to low
prob_sorted = prob_df.sort_values(by='probability (%)', ascending=False)[:-5]

In [39]:
# stations with less than 50 windspeed records
prob_less50 = prob_df[prob_df['count'] <50].sort_values(by='count', ascending=True)

In [40]:
def dist_boxplot(station):
    plt.figure(figsize =(4, 4))
    subset = np.array(windspeed_2023_08_02[windspeed_2023_08_02['station'] == station]['wind_speed'])
    sns.boxplot(subset, width=0.2)
    threshold = prob_df[prob_df['station'] == station]['threshold'].iloc[0]
    plt.axhline(threshold)
    prob = prob_df[prob_df['station'] == station]['probability (%)'].iloc[0]
    plt.text(x=0, y=38, s=f'probability: ' + str(prob), color='red')
    plt.title(station)
    plt.show()

In [ ]:
# can run a loop to show all stations distribtuion
# for station in station_codes:
#     dist_boxplot(station)

# showing an example for station "AMO"
dist_boxplot("AMO")

## 3. Geospatial Visualization


### 3.1 New datasets


In [ ]:
vri_df = pd.read_csv('data/src_vri_snapshot_2024_03_20.csv')
wingspan_df = pd.read_csv('data/dev_wings_agg_span_2024_01_01.csv')

C:\Users\glori\AppData\Local\Temp\ipykernel_15384\4187153652.py:4: DtypeWarning: Columns (3,4,5,6,8,10,11,12,14,16,17,20,21,22,23,24,25,34,35,36,37,38,42,44,45,65,66,67,68,69,71,72,82,83,84,87,88,91,92,94,100,101) have mixed types. Specify dtype option on import or set low_memory=False.
  wingspan_df = pd.read_csv('data/dev_wings_agg_span_2024_01_01.csv')


In [ ]:
vri_df.head()

In [ ]:
vri_df.columns

#### 3.1.1 Changing shape columns to geometry type

Currently, the `shape` column datatype is `str` when it should be geometry

Also need to reproject to the same `shape_srid` ESPG:4326

In [45]:
gis_2024_1004['shape'] = gpd.GeoSeries.from_wkt(gis_2024_1004['shape'])
gis_gdf = gpd.GeoDataFrame(gis_2024_1004, geometry='shape').set_crs(epsg=4431).to_crs(epsg=4326)
# gis_gdf.head()

In [46]:
vri_df['shape'] = gpd.GeoSeries.from_wkt(vri_df['shape'])
vri_gdf = gpd.GeoDataFrame(vri_df, geometry='shape').set_crs(epsg=4326)
# vri_gdf.head()

In [47]:
wingspan_df['shape'] = gpd.GeoSeries.from_wkt(wingspan_df['shape'])
wingspan_gdf = gpd.GeoDataFrame(wingspan_df, geometry='shape').set_crs(epsg=2230).to_crs(epsg=4326)
# wingspan_gdf.head()

In [48]:
# drop the shape_srid columns since we have reprojected and they are no longer correct/meaningful 
gis_gdf = gis_gdf.drop(columns=['shape_srid'])
vri_gdf = vri_gdf.drop(columns=['shape_srid'])
wingspan_gdf = wingspan_gdf.drop(columns=['shape_srid'])

#### 3.1.2 Merging datasets 

In [49]:
# merge on weather station codes, not yet spatial join using gpd
gis_vri_merge = gis_gdf.merge(vri_gdf, left_on='weatherstationcode', right_on='anemometercode')
# gis_vri_merge

In [50]:
# find polygon centroids then merge with points
vri_gdf['centroid'] = vri_gdf['shape'].centroid
# vri_gdf.head()

C:\Users\glori\AppData\Local\Temp\ipykernel_15384\997739813.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  vri_gdf['centroid'] = vri_gdf['shape'].centroid


In [51]:
# spatial join
vri_gis_sjoin = vri_gdf.sjoin(gis_gdf, how='inner')
# vri_gis_sjoin.head()

> Found some anomolies with the dataframe sizes, there seem to be duplicates with the same station name

In [52]:
gis_gdf.shape

(223, 26)

In [53]:
vri_gdf.shape

(308, 27)

In [54]:
vri_gis_sjoin.shape
# one extra row?

(309, 53)

In [55]:
vri_gis_sjoin.index.nunique()
# duplicates

253

In [56]:
# another spatial join
vri_wingspan_sjoin = vri_gdf.sjoin(wingspan_gdf)
# vri_wingspan_sjoin.head()

In [57]:
vri_wingspan_sjoin.shape

(255894, 130)

In [58]:
wingspan_gdf.shape
# significantly less rows (intersections)

(674592, 103)

### 3.2 Visualization with probabilities

#### 3.2.1 Folium map with different layers

Weather station markers, VRI risks (heatmap), VRI areas (polygons), PSPS probability (heatmap)

In [59]:
# merge prob_df with the new spatially joined df
prob_merge = vri_gis_sjoin.merge(prob_df, left_on='weatherstationcode', right_on='station').merge(station_summary_2023_08_02, left_on='weatherstationcode', right_on='station')
# prob_merge

In [60]:
## VRI risk heatmap
vri_weights = {'H': 3, 'M': 2, 'L': 1}
prob_merge['vri_weight'] = prob_merge['vri'].map(vri_weights)

## PSPS probability heatmap 
# FIXME: heatmap weights not displaying correctly
prob_quantiles = prob_merge['probability (%)'].quantile([0.25, 0.5, 0.75]).tolist()
prob_weights = []
for _, row in prob_merge.iterrows():
    w = 0
    if row['probability (%)'] < prob_quantiles[0]:
        w = 1
    elif row['probability (%)'] < prob_quantiles[1]:
        w = 2
    else:
        w = 3
    prob_weights.append(w)
prob_merge['psps_weight'] = prob_weights

# Check for missing values and remove rows with NaN in latitude, longitude, vri_weight, or probability
cleaned_df = prob_merge.dropna(subset=['latitude', 'longitude', 'vri_weight', 'psps_weight'])

# Create a list of [latitude, longitude, weight] for the heatmap
heat_data = [[row['latitude'], row['longitude'], row['vri_weight']] for index, row in cleaned_df.iterrows()]

# Create a folium map centered around the average coordinates of the data
middle_point = [cleaned_df['latitude'].mean(), cleaned_df['longitude'].mean()]
m = folium.Map(location=middle_point, zoom_start=10)

# Add the heatmap layers
heatmap_layer = folium.FeatureGroup(name='VRI risk')
HeatMap(heat_data, min_opacity=0.2, radius=20, blur=15, max_zoom=1, name='VRI risk').add_to(heatmap_layer)
heatmap_layer.add_to(m)

psps_prob = folium.FeatureGroup(name='PSPS probability')
heat_data2 = [[row['latitude'], row['longitude'], row['psps_weight']] for index, row in cleaned_df.iterrows()]
HeatMap(heat_data2, min_opacity=0.2, radius=20, blur=15, max_zoom=1, name='PSPS prob').add_to(psps_prob)
psps_prob.add_to(m)


## Add weather station points to the map
marker_group = folium.FeatureGroup(name="Weather stations")
for _, row in prob_merge.iterrows():
    # Create a popup with relevant information
    popup_text = f"""
    Weather Station: {row['weatherstationname']} ({row['weatherstationcode']})<br>
    Elevation: {row['elevation']} m<br>
    NWS Zone: {row['nwszone']}<br>
    PSPS Probability: {row['probability (%)']}<br>
    """
    
    # Add a marker for each weather station
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(marker_group)
marker_group.add_to(m)


## Add VRI polygons layer
vri_polygons = folium.FeatureGroup(name='VRI polygons')
for i in vri_gdf['shape']:
    folium.GeoJson(i).add_to(vri_polygons)
vri_polygons.add_to(m)


# Create a layer control object and add it to our map instance
folium.LayerControl().add_to(m)

# Save the map to an HTML file and display it
m.save('layered_map.html')

# Display interactive map in Jupyter
# m

#### 3.2.2 Conductor spans 

In [61]:
# wingspan_gdf.groupby(by='psps_station').count()
# each psps station has a different number of conductor spans

In [62]:
# create folium map object
conductor_map = folium.Map(location=middle_point)


## add weather station points to the map
marker_group = folium.FeatureGroup(name="Weather stations")
for _, row in gis_gdf.iterrows():
    # Create a popup with relevant information
    popup_text = f"""
    Weather Station: {row['weatherstationname']}<br>
    Structure ID: {row['structureid']}<br>
    """
    
    # Add a marker for each weather station
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(marker_group)
marker_group.add_to(conductor_map)


# add hlines (each line is blue and very short)
line_group = folium.FeatureGroup(name='Conductor spans')
# only using the first 1000 lines as examples so the map/file isn't too large
for i in wingspan_gdf['shape'][:1000]:
    folium.GeoJson(i).add_to(line_group)
line_group.add_to(conductor_map)

# add lines, grouped by the psps station it is tied to 
# commented out bc the full map/file becomes too large to be uploaded to github
# for group_name, group_data in wingspan_gdf.groupby('psps_station'):
#     feature_group = folium.FeatureGroup(name=str(group_name))
#     for _, row in group_data.iterrows()[:1000]:
#         folium.GeoJson(
#             row['shape'],
#             name=str(group_name)
#         ).add_to(feature_group)
#     feature_group.add_to(conductor_map)


# add layer (to show the difference of added objects more clearly)
folium.LayerControl().add_to(conductor_map)

# Save the map to an HTML file and display it
conductor_map.save('conductor_span_map.html')

# conductor_map

> Attempted to color each group of conductor spans differently but had some difficulties. The code in the cell below contains errors and is incomplete/does not display correctly.

In [63]:
# create folium map object
conductor_map = folium.Map(location=middle_point)

## add lines, grouped by the psps station it is tied to 

# define a style function for different stations/conductor groups
def style_function(group_name):
    group_name = str(group_name)
    if ord(group_data[0]) < 71:
        return {'color': 'orange', 'opacity':0.5}
    elif ord(group_data[0]) < 80:
        return {'color': 'pink', 'opacity':0.5}
    else:
        return {'color': 'purple', 'opacity':0.5}
    
for group_name, group_data in wingspan_gdf.groupby('psps_station'):
    feature_group = folium.FeatureGroup(name=str(group_name))
    for _, row in group_data.iterrows():
        print(row['shape'])
        folium.GeoJson(row['shape'], name=str(group_name), style_function=style_function).add_to(feature_group)
    feature_group.add_to(conductor_map)

# add layer control 
folium.LayerControl().add_to(conductor_map)

# conductor_map

LINESTRING (-117.2489844152717 33.36251395510124, -117.2490227915011 33.36251409901239, -117.2490305309156 33.36252238895723, -117.2493242214726 33.36252357354851, -117.2493327467301 33.36252254813497, -117.2493410980238 33.3625207778886, -117.2493491826715 33.36251828649861, -117.2493569127012 33.36251509685241, -117.2493642026199 33.36251124986617, -117.2493709724645 33.36250678394365, -117.2493771516099 33.36250174757975, -117.2493826672817 33.36249619747974, -117.2493874634308 33.36249019311119, -117.2493914825912 33.36248380348449, -117.2493946859549 33.36247709262277, -117.2493958060387 33.3624741219845, -117.2494006615879 33.36246553168967, -117.249406563383 33.36245741277092, -117.2494134406315 33.362449858397, -117.2494212247157 33.36244294627029, -117.2494237380283 33.36244083326219, -117.2494313326589 33.36243529432124, -117.249439573983 33.36243045217067, -117.2494483721206 33.36242636103945, -117.2494576326338 33.36242306092624, -117.2494672544047 33.36242059283163, -117.2

KeyError: 0